## Using Komo for IK

In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/03-motion'** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '


**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/03-motion'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
# RealWorld <- Attach the simulator

# C is for plannning
#   plan -> q (afterwards give it to Simulator)
#   you can set the joints as you want here, but accessing the simulator with the q function
#   if you give q to the C, it will just change the configuration
#   if you give q to the RealWorld, it will do all the calculations (colisions and so on, and we can see how it relates to other opjects)
#   in RealWorld you can model how object moves ...

# If you update the simulator, then it updates the RealWorld 
# 

In [3]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

In [4]:
obj = C.addFrame("object")
obj.setPosition([1., 0., 1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

In [5]:
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionDiff, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,.0])

In [6]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()
C.setFrameState( IK.getFrameState(0) )

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000872 (kin:4.3e-05 coll:0 feat:0.000334 newton: 0.000122) setJointStateCount:9
   sos:2.01047 ineq:0 eq:0.00280914


In [7]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# # copy C into the IK's internal configuration(s)
# IK.setConfigurations(C)

# redefine the IK problem
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.1]);
# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )

{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 1.2722238184459724}, 'F_qQuaternionNorms/0-#110': {'order': 0.0, 'type': 'eq'}, 'F_PositionRel/0-object-R_gripperCenter': {'order': 0.0, 'type': 'eq', 'eq': 0.00032602403429674714}, 'sos': 1.2849460566304323, 'ineq': 0.0, 'eq': 0.00032602403429674714, 'f': 0.0}** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.00225 (kin:4.8e-05 coll:0 feat:0.000476 newton: 0.001346) setJointStateCount:11
   sos:1.28495 ineq:0 eq:0.000326024



## Path Optimization

In [5]:
obj.setPosition([.8,.2,1.5])



In [5]:
# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
# parameters (phase, step per phase, duration, collision checking)
komo = C.komo_path(1.,40, 5., False)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=2) # At the end velocity is zero
# komo.addObjective([1.], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2])


In [6]:
komo.optimize()


** KOMO::run solver:sparse collisions:0 x-dim:560 T:40 k:2 phases:1 stepsPerPhase:40 tau:0.125  #timeSlices:42 #totalDOFs:560 #frames:4620
** optimization time:0.024391 (kin:0.001897 coll:0 feat:0.012219 newton: 0.004924) setJointStateCount:20
   sos:0.0734672 ineq:0 eq:9.80067e-08


In [7]:
komo.getReport()

{'F_qItself/2-#28': {'order': 2.0, 'type': 'eq', 'eq': 9.80066625078102e-08},
 'F_PositionDiff/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'sos',
  'sos': 4.230258504353078e-05},
 'sos': 0.07346718652220179,
 'ineq': 0.0,
 'eq': 9.80066625078102e-08,
 'f': 0.0}

In [ ]:
while True:
    V = komo.view_play(True, 0.1)

## Check Collisions

In [ ]:
C.setFrameState(komo.getFrameState(19))
C.getJointState()
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)